# LogP Prediction using TPOT AutoML
## Advanced Pipeline Optimization for Experimental LogP Prediction

This notebook uses TPOT (Tree-based Pipeline Optimization Tool) to automatically find the best machine learning pipeline for predicting experimental logP from calculated values.

**Goals:**
- Improve upon 80% linear regression accuracy
- Remove 7 identified outliers (SM28, SM32, SM33, SM35, SM36, SM41, SM42)
- Engineer meaningful molecular descriptors
- Apply proper cross-validation to prevent overfitting
- Use TPOT to find optimal preprocessing + model pipeline

## 1. Setup and Installation

In [ ]:
# Install required packages (uncomment if running in fresh Colab environment)
!pip install tpot scikit-learn pandas numpy matplotlib seaborn rdkit

# Note: If rdkit installation fails in Colab, use:
# !pip install rdkit-pypi

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 2. Load and Prepare Data

In [ ]:
# Upload the CSV file in Colab
# Method 1: Upload from local machine
from google.colab import files
uploaded = files.upload()

# Method 2: If you prefer, mount Google Drive and read from there
# from google.colab import drive
# drive.mount('/content/drive')
# df = pd.read_csv('/content/drive/MyDrive/path/to/logP_data_from_ods.csv')

In [ ]:
# Load the data
df = pd.read_csv('logP_data_from_ods.csv')

print(f"Dataset shape: {df.shape}")
print("\nFirst few rows:")
df.head()

In [ ]:
# Check for missing values and data info
print("\nDataset Info:")
print(df.info())
print("\nMissing values per column:")
print(df.isnull().sum())
print("\nBasic statistics:")
df.describe()

## 3. Remove Outliers and Clean Data

In [ ]:
# Define outliers to remove
outliers = ['SM28', 'SM32', 'SM33', 'SM35', 'SM36', 'SM41', 'SM42']

print(f"Original dataset size: {len(df)}")
print(f"Outliers to remove: {outliers}")

# Remove outliers
df_clean = df[~df['MNSol_id'].isin(outliers)].copy()

print(f"Dataset size after removing outliers: {len(df_clean)}")
print(f"Removed {len(df) - len(df_clean)} samples")

In [ ]:
# Remove rows with missing Exp_logP values (our target variable)
df_clean = df_clean.dropna(subset=['Exp_logP'])

print(f"Dataset size after removing missing targets: {len(df_clean)}")
print(f"\nRemaining columns: {list(df_clean.columns)}")

## 4. Feature Engineering: Create Meaningful Descriptors

Beyond the simple BAR_logP, we'll create additional features that capture important molecular properties:
- Energy differences and ratios
- Partition behavior indicators
- Non-linear transformations
- Interaction terms

In [ ]:
def engineer_features(df):
    """
    Engineer meaningful molecular descriptors from BAR calculations
    """
    df_feat = df.copy()
    
    # Original features
    df_feat['BAR_logP'] = df['BAR_logP']
    df_feat['BAR_Water'] = df['BAR_Water']
    df_feat['BAR_Octanol'] = df['BAR_Octanol']
    
    # 1. Energy-based descriptors
    # Total solvation energy
    df_feat['Total_Solvation_Energy'] = df['BAR_Water'] + df['BAR_Octanol']
    
    # Absolute energy difference (partition strength)
    df_feat['Energy_Difference_Abs'] = np.abs(df['BAR_Water'] - df['BAR_Octanol'])
    
    # Solvation ratio (avoids division by zero)
    df_feat['Solvation_Ratio'] = df['BAR_Water'] / (df['BAR_Octanol'] + 1e-6)
    
    # 2. Non-linear transformations
    # These can capture non-linear relationships
    df_feat['BAR_logP_squared'] = df['BAR_logP'] ** 2
    df_feat['BAR_logP_cubed'] = df['BAR_logP'] ** 3
    df_feat['BAR_logP_sqrt'] = np.sqrt(np.abs(df['BAR_logP']))  * np.sign(df['BAR_logP'])
    
    # Log transformations (shifted to avoid log(0))
    df_feat['log_BAR_Water'] = np.log(np.abs(df['BAR_Water']) + 1) * np.sign(df['BAR_Water'])
    df_feat['log_BAR_Octanol'] = np.log(np.abs(df['BAR_Octanol']) + 1) * np.sign(df['BAR_Octanol'])
    
    # 3. Partition behavior indicators
    # Hydrophilicity indicator
    df_feat['Hydrophilic_Score'] = df['BAR_Water'] / (np.abs(df['BAR_Water']) + np.abs(df['BAR_Octanol']) + 1e-6)
    
    # Lipophilicity indicator
    df_feat['Lipophilic_Score'] = np.abs(df['BAR_Octanol']) / (np.abs(df['BAR_Water']) + np.abs(df['BAR_Octanol']) + 1e-6)
    
    # 4. Interaction terms
    df_feat['Water_Octanol_Product'] = df['BAR_Water'] * df['BAR_Octanol']
    
    # 5. Linear correction from your Excel formula: y = 0.8645x - 0.1688
    df_feat['Linear_Corrected_logP'] = 0.8645 * df['BAR_logP'] - 0.1688
    
    # 6. Deviation from linear prediction
    df_feat['Linear_Deviation'] = df['BAR_logP'] - df_feat['Linear_Corrected_logP']
    
    # 7. Energy normalized by logP
    df_feat['Energy_per_logP'] = df_feat['Total_Solvation_Energy'] / (np.abs(df['BAR_logP']) + 1e-6)
    
    # 8. Categorize molecules by water solubility
    df_feat['High_Water_Solubility'] = (df['BAR_Water'] > df['BAR_Water'].median()).astype(int)
    df_feat['High_Octanol_Solubility'] = (df['BAR_Octanol'] < df['BAR_Octanol'].median()).astype(int)
    
    return df_feat

# Apply feature engineering
df_features = engineer_features(df_clean)

print(f"Number of engineered features: {len(df_features.columns)}")
print(f"\nNew features created:")
new_features = [col for col in df_features.columns if col not in df_clean.columns]
for i, feat in enumerate(new_features, 1):
    print(f"{i}. {feat}")

## 5. Prepare Features and Target for Modeling

In [ ]:
# Select features for modeling
feature_columns = [col for col in df_features.columns 
                   if col not in ['MNSol_id', 'Name', 'Exp_Water', 'Exp_Octanol', 'Exp_logP']]

X = df_features[feature_columns].values
y = df_features['Exp_logP'].values

print(f"Feature matrix shape: {X.shape}")
print(f"Target vector shape: {y.shape}")
print(f"\nFeatures used for modeling:")
for i, feat in enumerate(feature_columns, 1):
    print(f"{i}. {feat}")

In [ ]:
# Check for any remaining NaN or infinite values
print("Checking for invalid values...")
print(f"NaN values in X: {np.isnan(X).sum()}")
print(f"Infinite values in X: {np.isinf(X).sum()}")
print(f"NaN values in y: {np.isnan(y).sum()}")

# Replace any inf values with large finite numbers
X = np.nan_to_num(X, nan=0.0, posinf=1e6, neginf=-1e6)

print("\nData cleaning complete!")

## 6. Split Data for Training and Testing

In [ ]:
# Split data: 80% training, 20% testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Test set proportion: {X_test.shape[0] / len(X) * 100:.1f}%")

## 7. Baseline: Linear Regression Benchmark

In [ ]:
# Fit simple linear regression as baseline
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline

# Create baseline pipeline with scaling + ridge regression
baseline_model = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Ridge(alpha=1.0))
])

baseline_model.fit(X_train, y_train)

# Predictions
y_pred_baseline = baseline_model.predict(X_test)

# Evaluate
r2_baseline = r2_score(y_test, y_pred_baseline)
rmse_baseline = np.sqrt(mean_squared_error(y_test, y_pred_baseline))
mae_baseline = mean_absolute_error(y_test, y_pred_baseline)

print("="*60)
print("BASELINE MODEL PERFORMANCE (Ridge Regression)")
print("="*60)
print(f"R² Score: {r2_baseline:.4f} ({r2_baseline*100:.2f}% accuracy)")
print(f"RMSE: {rmse_baseline:.4f}")
print(f"MAE: {mae_baseline:.4f}")
print("="*60)

# Cross-validation score
cv_scores = cross_val_score(baseline_model, X_train, y_train, cv=5, 
                            scoring='r2', n_jobs=-1)
print(f"\nCross-validation R² scores: {cv_scores}")
print(f"Mean CV R²: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

## 8. TPOT: Automated Machine Learning Pipeline Optimization

TPOT will automatically:
- Try different preprocessing methods (scaling, feature selection, PCA, etc.)
- Test multiple regression algorithms (Random Forest, XGBoost, SVR, etc.)
- Optimize hyperparameters
- Find the best pipeline combination

In [ ]:
from tpot import TPOTRegressor

   "# Configure TPOT for comprehensive search\n",
    "# MAXIMUM THOROUGHNESS SETTINGS for best possible results\n",
    "# This will take 6-8 hours but will find the absolute best pipeline\n",
    "\n",
    "tpot_config = {\n",
    "    'generations': 150,  # Maximum generations for exhaustive exploration\n",
    "    'population_size': 100,  # Large population for maximum diversity\n",
    "    'cv': 5,  # 5-fold cross-validation (good for ~100 samples)\n",
    "    'random_state': 42,\n",
    "    'verbosity': 2,  # Show progress\n",
    "    'scoring': 'r2',  # Optimize for R² score\n",
    "    'n_jobs': -1,  # Use all available cores\n",
    "    'max_time_mins': 480,  # 8 hours - maximum thoroughness\n",
    "    'max_eval_time_mins': 10,  # Allow more time for complex pipelines\n",
    "    'early_stop': 20,  # More patience before stopping (20 generations)\n",
    "}"

print("TPOT Configuration:")
print("="*60)
for key, value in tpot_config.items():
    print(f"{key}: {value}")
    "print(\"=\"*60)\n",
    "print(\"\\n🚀 MAXIMUM THOROUGHNESS MODE ACTIVATED!\")\n",
    "print(\"Expected runtime: ~6-8 hours (480 minutes max)\")\n",
    "print(\"This will evaluate 15,000+ pipeline configurations\")\n",
    "print(\"With 5-fold CV: ~75,000 total model fits\")\n",
    "print(\"\\n⚠️  IMPORTANT: Make sure Google Colab won't timeout!\")\n",
    "print(\"Tip: Keep the browser tab active or use Colab Pro for longer runtimes\")\n",
    "print(\"\\nFor faster testing: reduce to generations=20, population_size=20, max_time_mins=30\")\n",
    "print(\"Current settings will find the ABSOLUTE BEST pipeline!\\n\")"

In [ ]:
# Initialize TPOT
tpot = TPOTRegressor(**tpot_config)

print("Starting TPOT optimization...")
print("This may take a while. Please be patient!\n")

# Fit TPOT
tpot.fit(X_train, y_train)

print("\n" + "="*60)
print("TPOT OPTIMIZATION COMPLETE!")
print("="*60)

## 9. Evaluate TPOT Model Performance

In [ ]:
# Make predictions with TPOT model
y_pred_tpot = tpot.predict(X_test)

# Calculate metrics
r2_tpot = r2_score(y_test, y_pred_tpot)
rmse_tpot = np.sqrt(mean_squared_error(y_test, y_pred_tpot))
mae_tpot = mean_absolute_error(y_test, y_pred_tpot)

print("="*60)
print("TPOT MODEL PERFORMANCE")
print("="*60)
print(f"R² Score: {r2_tpot:.4f} ({r2_tpot*100:.2f}% accuracy)")
print(f"RMSE: {rmse_tpot:.4f}")
print(f"MAE: {mae_tpot:.4f}")
print("="*60)

# Compare with baseline
print("\n" + "="*60)
print("COMPARISON: TPOT vs BASELINE")
print("="*60)
print(f"R² Improvement: {(r2_tpot - r2_baseline):.4f} ({(r2_tpot - r2_baseline)*100:.2f}%)")
print(f"RMSE Improvement: {(rmse_baseline - rmse_tpot):.4f}")
print(f"MAE Improvement: {(mae_baseline - mae_tpot):.4f}")
print("="*60)

if r2_tpot > r2_baseline:
    print("\n✅ TPOT model outperforms baseline!")
else:
    print("\n⚠️  Baseline still competitive. Consider longer TPOT runtime.")

## 10. Cross-Validation Analysis (Preventing Overfitting)

In [ ]:
# Perform detailed cross-validation analysis
from sklearn.model_selection import cross_validate

cv_results = cross_validate(
    tpot.fitted_pipeline_,  # Use the best pipeline found by TPOT
    X_train, 
    y_train,
    cv=5,
    scoring=['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'],
    return_train_score=True,
    n_jobs=-1
)

print("="*60)
print("5-FOLD CROSS-VALIDATION RESULTS")
print("="*60)

# R² scores
train_r2 = cv_results['train_r2']
test_r2 = cv_results['test_r2']

print(f"\nR² Score:")
print(f"  Training:   {train_r2.mean():.4f} (+/- {train_r2.std() * 2:.4f})")
print(f"  Validation: {test_r2.mean():.4f} (+/- {test_r2.std() * 2:.4f})")

# RMSE
train_rmse = np.sqrt(-cv_results['train_neg_mean_squared_error'])
test_rmse = np.sqrt(-cv_results['test_neg_mean_squared_error'])

print(f"\nRMSE:")
print(f"  Training:   {train_rmse.mean():.4f} (+/- {train_rmse.std() * 2:.4f})")
print(f"  Validation: {test_rmse.mean():.4f} (+/- {test_rmse.std() * 2:.4f})")

# MAE
train_mae = -cv_results['train_neg_mean_absolute_error']
test_mae = -cv_results['test_neg_mean_absolute_error']

print(f"\nMAE:")
print(f"  Training:   {train_mae.mean():.4f} (+/- {train_mae.std() * 2:.4f})")
print(f"  Validation: {test_mae.mean():.4f} (+/- {test_mae.std() * 2:.4f})")

# Overfitting check
overfit_indicator = train_r2.mean() - test_r2.mean()
print(f"\n{'='*60}")
print(f"Overfitting Indicator: {overfit_indicator:.4f}")
if overfit_indicator < 0.1:
    print("✅ Model generalizes well! Low overfitting.")
elif overfit_indicator < 0.2:
    print("⚠️  Moderate overfitting detected.")
else:
    print("❌ High overfitting! Model may not generalize well.")
print("="*60)

## 11. Visualize Results

In [ ]:
# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Predicted vs Actual (TPOT)
ax1 = axes[0, 0]
ax1.scatter(y_test, y_pred_tpot, alpha=0.6, s=100, edgecolors='k', linewidth=0.5)
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Perfect Prediction')
ax1.set_xlabel('Experimental logP', fontsize=12, fontweight='bold')
ax1.set_ylabel('Predicted logP (TPOT)', fontsize=12, fontweight='bold')
ax1.set_title(f'TPOT Model: R² = {r2_tpot:.4f}', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# 2. Predicted vs Actual (Baseline)
ax2 = axes[0, 1]
ax2.scatter(y_test, y_pred_baseline, alpha=0.6, s=100, 
            edgecolors='k', linewidth=0.5, color='orange')
ax2.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
         'r--', lw=2, label='Perfect Prediction')
ax2.set_xlabel('Experimental logP', fontsize=12, fontweight='bold')
ax2.set_ylabel('Predicted logP (Baseline)', fontsize=12, fontweight='bold')
ax2.set_title(f'Baseline Model: R² = {r2_baseline:.4f}', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

# 3. Residuals Plot (TPOT)
ax3 = axes[1, 0]
residuals_tpot = y_test - y_pred_tpot
ax3.scatter(y_pred_tpot, residuals_tpot, alpha=0.6, s=100, edgecolors='k', linewidth=0.5)
ax3.axhline(y=0, color='r', linestyle='--', lw=2)
ax3.set_xlabel('Predicted logP', fontsize=12, fontweight='bold')
ax3.set_ylabel('Residuals', fontsize=12, fontweight='bold')
ax3.set_title('TPOT Residuals Plot', fontsize=14, fontweight='bold')
ax3.grid(True, alpha=0.3)

# 4. Model Comparison
ax4 = axes[1, 1]
models = ['Baseline\n(Ridge)', 'TPOT\n(Optimized)']
r2_scores = [r2_baseline, r2_tpot]
colors = ['orange', 'steelblue']

bars = ax4.bar(models, r2_scores, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
ax4.set_ylabel('R² Score', fontsize=12, fontweight='bold')
ax4.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax4.set_ylim([0, 1])
ax4.axhline(y=0.8, color='green', linestyle='--', lw=2, label='80% Target')
ax4.legend(fontsize=10)

# Add value labels on bars
for bar, score in zip(bars, r2_scores):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
             f'{score:.4f}\n({score*100:.2f}%)',
             ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.savefig('logP_prediction_results.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n📊 Visualization saved as 'logP_prediction_results.png'")

## 12. Export Optimized Pipeline

In [ ]:
# Export the best pipeline as Python code
tpot.export('tpot_logP_pipeline.py')

print("✅ Best pipeline exported to 'tpot_logP_pipeline.py'")
print("\nYou can use this pipeline in production without TPOT!\n")

# Display the pipeline
print("="*60)
print("BEST PIPELINE FOUND BY TPOT:")
print("="*60)
print(tpot.fitted_pipeline_)
print("="*60)

## 13. Feature Importance Analysis

In [ ]:
# Try to extract feature importances if available
try:
    # Get the final estimator from the pipeline
    final_estimator = None
    
    # Check if fitted_pipeline has named_steps (sklearn Pipeline)
    if hasattr(tpot.fitted_pipeline_, 'named_steps'):
        steps = list(tpot.fitted_pipeline_.named_steps.values())
        final_estimator = steps[-1]
    elif hasattr(tpot.fitted_pipeline_, 'steps'):
        final_estimator = tpot.fitted_pipeline_.steps[-1][1]
    
    # Extract feature importances
    if final_estimator is not None and hasattr(final_estimator, 'feature_importances_'):
        importances = final_estimator.feature_importances_
        
        # Create dataframe
        feature_importance_df = pd.DataFrame({
            'Feature': feature_columns,
            'Importance': importances
        }).sort_values('Importance', ascending=False)
        
        print("\n" + "="*60)
        print("FEATURE IMPORTANCE RANKING")
        print("="*60)
        print(feature_importance_df.to_string(index=False))
        print("="*60)
        
        # Plot top 15 features
        plt.figure(figsize=(12, 8))
        top_features = feature_importance_df.head(15)
        plt.barh(range(len(top_features)), top_features['Importance'], 
                color='steelblue', edgecolor='black', linewidth=1)
        plt.yticks(range(len(top_features)), top_features['Feature'])
        plt.xlabel('Importance', fontsize=12, fontweight='bold')
        plt.title('Top 15 Most Important Features', fontsize=14, fontweight='bold')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        print("\n📊 Feature importance plot saved as 'feature_importance.png'")
        
    else:
        print("\n⚠️  Selected model doesn't support feature importance extraction.")
        print("This is common for some models like SVR or neural networks.")
        
except Exception as e:
    print(f"\n⚠️  Could not extract feature importances: {str(e)}")
    print("This is normal for some pipeline configurations.")

## 14. Save Model for Future Use

In [ ]:
import joblib

# Save the entire TPOT fitted pipeline
joblib.dump(tpot.fitted_pipeline_, 'tpot_logP_model.pkl')

print("✅ Model saved as 'tpot_logP_model.pkl'")
print("\nTo load and use the model later:")
print("""\n# Load model
import joblib
model = joblib.load('tpot_logP_model.pkl')

# Make predictions
predictions = model.predict(X_new)
""")

## 15. Make Predictions on New Data (Example)

In [ ]:
# Example: Create predictions for all data
X_all = df_features[feature_columns].values
X_all = np.nan_to_num(X_all, nan=0.0, posinf=1e6, neginf=-1e6)
y_all_pred = tpot.fitted_pipeline_.predict(X_all)

# Add predictions to dataframe
df_results = df_features[['MNSol_id', 'Name', 'BAR_logP', 'Exp_logP']].copy()
df_results['Predicted_logP'] = y_all_pred
df_results['Prediction_Error'] = df_results['Exp_logP'] - df_results['Predicted_logP']
df_results['Absolute_Error'] = np.abs(df_results['Prediction_Error'])

# Sort by absolute error
df_results_sorted = df_results.sort_values('Absolute_Error')

print("\n" + "="*80)
print("BEST PREDICTIONS (Top 10 with lowest error)")
print("="*80)
print(df_results_sorted.head(10).to_string(index=False))

print("\n" + "="*80)
print("WORST PREDICTIONS (Top 10 with highest error)")
print("="*80)
print(df_results_sorted.tail(10).to_string(index=False))

# Save results
df_results_sorted.to_csv('logP_predictions_detailed.csv', index=False)
print("\n✅ Detailed predictions saved to 'logP_predictions_detailed.csv'")

## 16. Summary and Recommendations

In [ ]:
print("\n" + "="*80)
print("FINAL SUMMARY AND RECOMMENDATIONS")
print("="*80)

print(f"\n📊 PERFORMANCE METRICS:")
print(f"   Baseline R²: {r2_baseline:.4f} ({r2_baseline*100:.2f}%)")
print(f"   TPOT R²:     {r2_tpot:.4f} ({r2_tpot*100:.2f}%)")
print(f"   Improvement: {(r2_tpot - r2_baseline)*100:.2f}%")

print(f"\n🎯 TARGET ACHIEVEMENT:")
if r2_tpot >= 0.80:
    print(f"   ✅ SUCCESS! Exceeded 80% accuracy target")
    print(f"   Achieved: {r2_tpot*100:.2f}%")
else:
    print(f"   ⚠️  Close to target: {r2_tpot*100:.2f}%")
    print(f"   Need: {(0.80-r2_tpot)*100:.2f}% more")

print(f"\n🔬 DATA QUALITY:")
print(f"   Total samples: {len(df_clean)}")
print(f"   Training samples: {len(X_train)}")
print(f"   Test samples: {len(X_test)}")
print(f"   Features engineered: {len(feature_columns)}")

print(f\"\\n📈 RECOMMENDATIONS FOR FURTHER IMPROVEMENT:\")\n",
    "print(\"\"\"\\n   1. ✅ MAXIMUM TPOT (150 gen, 100 pop, 480 min) - Already configured!\n",
    "   2. ✅ This is the most thorough configuration possible!\n",
    "   3. If results still need improvement, consider:\n",
    "      • Ensemble methods (stacking multiple TPOT runs with different seeds)\n",
    "      • Add more chemical descriptors (molecular weight, rotatable bonds,\n",
    "        TPSA, aromatic rings, H-bond donors/acceptors, LogS, etc.)\n",
    "      • Use RDKit to calculate 200+ molecular descriptors from SMILES\n",
    "      • Collect more training data (more samples = better generalization)\n",
    "   4. Try different search spaces after this run:\n",
    "      • 'gradient-boosting' for tree-based models\n",
    "      • 'neural-network' for deep learning approaches\n",
    "   5. Investigate systematic errors in predictions\n",
    "   6. Consider domain-specific corrections based on chemical classes\n",
    "\"\"\")

print("="*80)
print("\n✅ Analysis complete! All results and models saved.")
print("="*80)

## Optional: Advanced TPOT Configuration

If you want to further optimize, you can try these advanced configurations:

In [ ]:
# Advanced TPOT configuration with custom search space
# Uncomment and run this cell if you want to try different configurations

"""
from tpot import TPOTRegressor

# Option 1: Use a lighter search space for faster optimization
tpot_light = TPOTRegressor(
    search_space='linear-light',  # Focus on linear models
    generations=30,
    population_size=30,
    cv=5,
    random_state=42,
    verbosity=2,
    n_jobs=-1,
    max_time_mins=30
)

# Option 2: Use gradient boosting models
tpot_gradient = TPOTRegressor(
    search_space='gradient-boosting',  # Focus on gradient boosting
    generations=40,
    population_size=40,
    cv=5,
    random_state=42,
    verbosity=2,
    n_jobs=-1,
    max_time_mins=45
)

# Option 3: Full search with longer runtime (recommended for best results)
tpot_full = TPOTRegressor(
    generations=100,
    population_size=100,
    cv=10,  # More folds for better validation
    random_state=42,
    verbosity=2,
    n_jobs=-1,
    max_time_mins=240,  # 4 hours
    early_stop=15
)
"""

print("Advanced configurations available in comments above.")
print("Uncomment and run to try different TPOT strategies.")